# <h1 style='background-color:LimeGreen; font-family:newtimeroman; font-size:200%; text-align:center; border-radius: 15px 50px;' >MobileNet </h1> 

MobileNet is a streamlined architecture that uses depthwise separable convolutions to construct lightweight deep convolutional neural networks and provides an efficient model for mobile and embedded vision applications . The structure of MobileNet is based on depthwise separable filters.

<img src="https://slideplayer.com/slide/16902656/97/images/10/MobileNet+Layer+Architecture.jpg" width="800px">



# <h1 style='background-color:LimeGreen; font-family:newtimeroman; font-size:180%; text-align:center; border-radius: 15px 50px;' > Data Description </h1>
Can you help detect farmers detect apple diseases? This competition builds on last year's by challenging you to handle additional diseases and to provide more detailed information about leaves that have multiple infections.

## Files
train.csv - the training set metadata.

* image - the image ID.

* labels - the target classes, a space delimited list of all diseases found in the image. Unhealthy leaves with too many diseases to classify visually will have the complex class, and may also have a subset of the diseases identified.


#### sample_submission.csv - A sample submission file in the correct format.

* image

* labels

train_images - The training set images.

test_images - The test set images. This competition has a hidden test set: only three images are provided here as samples while the remaining 5,000 images will be available to your notebook once it is submitted.

#### Dataset Link 

[Here](https://www.kaggle.com/c/plant-pathology-2021-fgvc8/data)

### Import the Libraries

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import PIL
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
from tqdm import tqdm
import tensorflow_addons as tfa
import random
from sklearn.preprocessing import MultiLabelBinarizer

pd.set_option("display.max_columns", None)

In [ ]:
train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
print(len(train))
print(train.columns)
print("Number of unique labels: ",len(train.labels.value_counts()))
print(train['labels'].value_counts().plot.bar())
print(train.labels.value_counts())
train.head()

In [ ]:
import pandas_profiling as pp
pp.ProfileReport(train)

In [ ]:
## If u choose the some lables 
#train['labels'] = train['labels'].apply(lambda string: string.split(' '))
#train

In [ ]:
##s = list(train['labels'])
##mlb = MultiLabelBinarizer()
#trainx = pd.DataFrame(mlb.fit_transform(s), columns=mlb.classes_, index=train.index)
#print(trainx.columns)
#print(trainx.sum())

#labels = list(trainx.sum().keys())
#print(labels)
#label_counts = trainx.sum().values.tolist()

#fig, ax = plt.subplots(1,1, figsize=(20,6))

#sns.barplot(x= labels, y= label_counts, ax=ax)

In [ ]:
fig1 = plt.figure(figsize=(26,10))

for i in range(1, 13):
    
    rand =  random.randrange(1, 18000)
    sample = os.path.join('../input/plant-pathology-2021-fgvc8/train_images/', train['image'][rand])
    
    img = PIL.Image.open(sample)
    
    ax = fig1.add_subplot(4,4,i)
    ax.imshow(img)
    
    title = f"{train['labels'][rand]}{img.size}"
    plt.title(title)
    
    fig1.tight_layout()


In [ ]:
%%time
datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
                                                        preprocessing_function=None,
                                                        data_format=None,
                                                    )

train_data = datagen.flow_from_dataframe(
    train,
    directory='../input/plant-pathology-2021-fgvc8/train_images',
    x_col="image",
    y_col= 'labels',
    color_mode="rgb",
    target_size = (150,150),
    class_mode="categorical",
    batch_size=32,
    shuffle=False,
    seed=40,
)



In [ ]:
seed = 40
tf.random.set_seed(seed)

model = tf.keras.applications.MobileNet(input_shape=(150,150,3),include_top=False,weights="imagenet")

print(model.input)
print(model.output)

In [ ]:
%%time
new_model = tf.keras.Sequential([
    model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(12, 
        kernel_initializer=keras.initializers.RandomUniform(seed=seed),
        bias_initializer=keras.initializers.Zeros(), name='dense', activation='sigmoid')
])

# Freezing the weights
for layer in new_model.layers[:-3]:
    layer.trainable=False
    
new_model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image
plot_model(new_model, to_file='convnet.png', show_shapes=True,show_layer_names=True)
Image(filename='convnet.png')

In [ ]:
f1 = tfa.metrics.F1Score(num_classes=12, average='macro')

callbacks = keras.callbacks.EarlyStopping(monitor=f1, patience=3, mode='max', restore_best_weights=True)


new_model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=keras.optimizers.Adam(lr=1e-3), 
              metrics= [f1])

new_model.fit(train_data, verbose = 1,epochs=15, callbacks=callbacks)

#### Submission


In [ ]:
test = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')

for img_name in tqdm(test['image']):
    path = '../input/plant-pathology-2021-fgvc8/test_images/'+str(img_name)
    with PIL.Image.open(path) as img:
        img = img.resize((150,150))
        img.save(f'./{img_name}')

In [ ]:
test_data = datagen.flow_from_dataframe(
    test,
    directory = './',
    x_col="image",
    y_col= None,
    color_mode="rgb",
    target_size = (150,150),
    classes=None,
    class_mode=None,
    batch_size=32,
    shuffle=False,
    seed=40,
)

preds = new_model.predict(test_data)
print(preds)
preds = preds.tolist()

indices = []
for pred in preds:
    temp = []
    for category in pred:
        if category>=0.3:
            temp.append(pred.index(category))
    if temp!=[]:
        indices.append(temp)
    else:
        temp.append(np.argmax(pred))
        indices.append(temp)
    
print(indices)

In [ ]:
labels = (train_data.class_indices)
labels = dict((v,k) for k,v in labels.items())
print(labels)

testlabels = []


for image in indices:
    temp = []
    for i in image:
        temp.append(str(labels[i]))
    testlabels.append(' '.join(temp))

print(testlabels)

In [ ]:
delfiles = tf.io.gfile.glob('./*.jpg')

for file in delfiles:
    os.remove(file)

In [ ]:
sub = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
sub['labels'] = testlabels
sub.to_csv('submission.csv', index=False)
sub